In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import sklearn
import boto3
from s3 import get_file
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolor

# Import Data from Amazon S3 into DataFrame

In [2]:
# connect to Amazon S3
s3 = boto3.resource('s3')
lyrics = get_file(s3,'s3ssp', download_file='NLP_Data/new_master_lyrics_audio_features.csv',rename_file='nlp.csv')

In [3]:
# create a pandas dataframe and drop 'na'
df = pd.read_csv(lyrics,sep='|',encoding='utf-8')
df_demo = df.copy().dropna()

In [4]:
# print top words in a topic
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)

In [5]:
# Remove more stopwords identified in pyLDAvis
df_demo['new_lyrics'] = df_demo['lyrics'].apply(lambda x: x.replace("wan", "")
                                                .replace("chorus", "")
                                                .replace("verse", "")
                                                .replace("gon", ""))

In [6]:
#create samples
df_test_one = df_demo.sample(1000)
df_test_two = df_demo.sample(3000)
df_test_three = df_demo.sample(5000)

In [7]:
#CountVectorizer hyperparameters 
max_df = .5
min_df = .005 

#LDA hyperparameters
n_topics = 30
n_words=10

In [8]:
#Fit CountVectorizer on lyrics data
vectorizer = CountVectorizer(analyzer='word',
                             min_df=min_df, max_df=max_df,     
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}')  # num chars > 3  

data_vectorized = vectorizer.fit_transform(df_test_two['new_lyrics'])

# Begin Topic Modeling

In [9]:
# lda model
lda_model = LatentDirichletAllocation(n_components=n_topics, learning_method="online",
                                max_iter=35, random_state=0, doc_topic_prior=.01)
lda_model.fit(data_vectorized)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=0.01,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=35, mean_change_tol=0.001,
             n_components=30, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

# Visualize Topics Spatially

In [10]:
# visualize n_topics spatially
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

/Applications/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
9     -65.023003  68.175766       1        1  10.264243
13    -62.640717   7.338716       2        1   7.696789
0     -47.685532  41.434364       3        1   7.205641
17     39.305195 -26.405161       4        1   6.425998
23    -39.370480 -12.702358       5        1   6.412770
6     -57.105282 -66.864906       6        1   6.148242
10    -21.085033  57.922813       7        1   5.719866
5      14.067936 -11.913107       8        1   5.276336
20    -33.067757  87.577042       9        1   4.170301
18      3.297307  84.915138      10        1   4.150056
3      66.044182  52.325340      11        1   3.991058
8      36.446163  73.225693      12        1   3.824607
27    -16.562317 -67.339020      13        1   3.495336
2      41.780869   4.328585      14        1   2.934699
22    -36.764111 -42.055058      15        1   2.798148
14     12.786169 -44.822781      16        1   2.632863
24     11.103065  51.155163      17        1   2.523530
16     38.065144  35.611355      18        1   2.432214
4     -11.234103   0.640888      19        1   2.291530
19     14.213376  17.692808      20        1   2.105822
29     71.903999 -19.578960      21        1   1.515937
15    -10.739471  30.960236      22        1   1.358145
26    -92.752235  -5.180922      23        1   1.212503
12     17.063776 -77.258751      24        1   1.082152
25     71.768265  16.681696      25        1   0.962007
7     -69.103088 -30.589602      26        1   0.419450
28    -82.763344  34.253414      27        1   0.349458
1      50.354053 -56.577091      28        1   0.305383
21    -34.611965  16.464033      29        1   0.221962
11    -11.234399 -28.530968      30        1   0.072953, topic_info=     Category         Freq      Term        Total  loglift  logprob
667   Default  3613.000000      love  3613.000000  30.0000  30.0000
1281  Default  1731.000000      yeah  1731.000000  29.0000  29.0000
406   Default  1751.000000      feel  1751.000000  28.0000  28.0000
41    Default  1311.000000      baby  1311.000000  27.0000  27.0000
1217  Default   640.000000      wait   640.000000  26.0000  26.0000
759   Default  1200.000000     night  1200.000000  25.0000  25.0000
270   Default  1457.000000       day  1457.000000  24.0000  24.0000
641   Default  1603.000000      life  1603.000000  23.0000  23.0000
473   Default  1153.000000      girl  1153.000000  22.0000  22.0000
662   Default   787.000000      lose   787.000000  21.0000  21.0000
431   Default   303.000000       fly   303.000000  20.0000  20.0000
1164  Default   496.000000   tonight   496.000000  19.0000  19.0000
262   Default   496.000000     dance   496.000000  18.0000  18.0000
651   Default   843.000000      live   843.000000  17.0000  17.0000
1156  Default  2308.000000      time  2308.000000  16.0000  16.0000
539   Default   435.000000       hey   435.000000  15.0000  15.0000
644   Default   862.000000     light   862.000000  14.0000  14.0000
114   Default   715.000000     break   715.000000  13.0000  13.0000
532   Default  1211.000000     heart  1211.000000  12.0000  12.0000
516   Default   741.000000      hand   741.000000  11.0000  11.0000
1062  Default   367.000000      star   367.000000  10.0000  10.0000
630   Default  1163.000000     leave  1163.000000   9.0000   9.0000
272   Default   378.000000      dead   378.000000   8.0000   8.0000
1003  Default   312.000000      sing   312.000000   7.0000   7.0000
724   Default   454.000000     money   454.000000   6.0000   6.0000
531   Default   777.000000      hear   777.000000   5.0000   5.0000
774   Default   337.000000       ooh   337.000000   4.0000   4.0000
386   Default   997.000000      fall   997.000000   3.0000   3.0000
547   Default   745.000000      hold   745.000000   2.0000   2.0000
661   Default   291.000000      lord   291.000000   1.0000   1.0000
...       ...          ...       ...          ...      ..

In [11]:
print("\nTopics in LDA model:")
cv_feature_names = vectorizer.get_feature_names()
print_top_words(lda_model, cv_feature_names, 30)


Topics in LDA model:
Topic #0: girl time talk bad sit guess friend watch road stay wrong pretty walk crazy day play reason care lot fine fun win leave woman eye remember read catch spend wear
Topic #1: fly settle apple snake leave stack ooh yeah singe rock glass worry touch silver roll ball sing hook plane guilty damn fuck dead teach feel king crowd stupid music life
Topic #2: night tonight dance ready rock party alive fight slow jump rhythm building wave floor revolution play thrill hit beat foot monkey hand house shut smile safe rescue spot time step
Topic #3: day life leave hard win bring late forget water wrong reach walk trouble fool climb hang meaning bell nightmare seek decide wise ready hold numb row boat shout ground meet
Topic #4: ooh hand breathe throw bang air shoot leave window woah play foot worth piece cross boom tall scared fake start truck game playin feel lean bare tired stand trigger finger
Topic #5: money shit nigga fuck bitch hit bout niggas baby light stick ass l

# Calculate probabilities and find dominant topics for each document

In [12]:
# Create Document — Topic Matrix

lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = [df_test_two['track_uri'].iloc[i] + str(i) for i in range(len(df_test_two))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Apply Style
df_document_topics = df_document_topic.sample(50).style.applymap(color_green).applymap(make_bold)
df_document_topics

In [13]:
# print documents by topic in order of probability
df_document_topic[['Topic1', 'dominant_topic']].sort_values(by='Topic1', ascending=False)

,Topic1,dominant_topic
7cg2YGhg1k6QoQdychuM3q1269,0.19,19
4fTg5dNKrfzuPs0oLku1Vl1755,0.19,5
659lYJRBHCq8cCKdxA2Z3f1409,0.15,9
4aUOG1Z1mDUajNQyIYUhfo2709,0.14,29
60tDcE1FnNDNgw0ykNYQQr58,0.14,4
7aokPpwjDPms1bKMWgKINz371,0.14,9
6kJeV9qTonZPGLijch4xzw2228,0.13,17
3TUzSZ8PZnLSiXMu5fsTNs783,0.13,6
2h02Nvz0YSL4usgTkgECVE161,0.12,6
5v2fnu64RUm3T4Ej17DeVk391,0.12,10
